In [17]:
from dotenv import load_dotenv
from pydantic import BaseModel
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
import os
from pprint import pprint

load_dotenv()


class ResearchResponse(BaseModel):
    topic: str
    summary: str
    sources: list[str]
    tools_used: list[str]


ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")

# llm = ChatOpenAI()
# llm2 = ChatAnthropic()
parser = PydanticOutputParser(pydantic_object=ResearchResponse)


prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            You are a research assistant that will help generate a research paper.
            Answer the user query and use neccessary tools. 
            Wrap the output in this format and provide no other text\n{format_instructions}
            """,
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{query}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
).partial(format_instructions=parser.get_format_instructions())
pprint(dict(prompt)["partial_variables"]["format_instructions"])

('The output should be formatted as a JSON instance that conforms to the JSON '
 'schema below.\n'
 '\n'
 'As an example, for the schema {"properties": {"foo": {"title": "Foo", '
 '"description": "a list of strings", "type": "array", "items": {"type": '
 '"string"}}}, "required": ["foo"]}\n'
 'the object {"foo": ["bar", "baz"]} is a well-formatted instance of the '
 'schema. The object {"properties": {"foo": ["bar", "baz"]}} is not '
 'well-formatted.\n'
 '\n'
 'Here is the output schema:\n'
 '```\n'
 '{"properties": {"topic": {"title": "Topic", "type": "string"}, "summary": '
 '{"title": "Summary", "type": "string"}, "sources": {"items": {"type": '
 '"string"}, "title": "Sources", "type": "array"}, "tools_used": {"items": '
 '{"type": "string"}, "title": "Tools Used", "type": "array"}}, "required": '
 '["topic", "summary", "sources", "tools_used"]}\n'
 '```')
